In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%aiida
from aiida.orm.data.base import Int, Float, Bool, Str, List
from aiida.orm.data.structure import StructureData
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.folder import FolderData
from aiida.work.run import submit

import ipywidgets as ipw
from IPython.display import display, clear_output, HTML
import nglview
import numpy as np

import tempfile
import shutil



import ase
import ase.io
from ase import Atom

from apps.surfaces.structure_browser import StructureBrowser
from apps.surfaces.widgets.computer_code_selection import ComputerCodeDropdown
from apps.surfaces.widgets.dft_details import DFTDetails
from apps.surfaces.widgets.neb_details import NEBDetails
from apps.surfaces.widgets.viewer_details import ViewerDetails
from apps.surfaces.widgets.slab_validity import slab_is_valid
from apps.surfaces.widgets.suggested_param import suggested_parameters
from apps.surfaces.widgets.submit_button import SubmitButton
from apps.surfaces.reactions.nebwork import NEBWorkchain

from ase.data import vdw_radii

from apps.surfaces.widgets import find_mol

In [ ]:
## GENERAL PARAMETERS
style = {'description_width': '120px'}
layout = {'width': '70%'}
the_workchain=NEBWorkchain
job_details = {'workchain':'NEBWorkchain'}
atoms=[]
details=[]
slab_analyzed=[]

In [ ]:
from tempfile import NamedTemporaryFile
from base64 import b64encode
def render_thumbnail(atoms):
    tmp = NamedTemporaryFile()
    ase.io.write(tmp.name, atoms, format='png') # does not accept StringIO
    raw = open(tmp.name).read()
    tmp.close()
    return b64encode(raw)

def display_thumbnail(th):
    return ipw.HTML('<img width="400px" src="data:image/png;base64,{}" title="">'.format(th))

In [ ]:
## WIDGETS MONITOR FUNCTIONS
def on_struct_change(c):
    global atoms,slab_analyzed
    if not struct_browser.results.value:
        return
    job_details['structure']=struct_browser.results.value
    node = struct_browser.results.value
    atoms = node.get_ase()
    atoms.pbc = [1, 1, 1]
    slab_analyzed = find_mol.analyze_slab(atoms)
    job_details['slab_analyzed']=slab_analyzed
    dft_details_widget.reset()  
    neb_details_widget.reset()
    #refresh_structure_view()
    guess_calc_params()
    
    viewer_widget.setup(atoms, slab_analyzed)
    
    with mol_ids_info_out:
        clear_output()
        print(slab_analyzed['summary'])

def on_fixed_atoms_btn_click(c):
    if dft_details_widget.btn_fixed_pressed:
        viewer_widget.show_fixed(dft_details_widget.fixed_atoms.value)
    else:
        viewer_widget.show_fixed("")
        
def guess_calc_params():
    method=dft_details_widget.calc_type.value
    valid_slab,msg = slab_is_valid(slab_analyzed,method)
    if valid_slab:        
        atoms_to_fix,num_nodes=suggested_parameters(slab_analyzed,method)
        dft_details_widget.reset(fixed_atoms=atoms_to_fix,num_machines=num_nodes,calc_type=method)
        #dft_details_widget.fixed_atoms.value = atoms_to_fix
        #dft_details_widget.num_nodes.value = num_nodes
    else:
        print(msg)

# Submit a NEB calculation

## Structure

In [ ]:
## DISPLAY WIDGETS AND DEFINE JOB PARAMETERS

##CODE
computer_code_dropdown = ComputerCodeDropdown(job_details=job_details,input_plugin='cp2k')

##STRUCTURE
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value') 

##DFT
dft_details_widget = DFTDetails(job_details=job_details)    
dft_details_widget.btn_fixed_atoms.on_click(on_fixed_atoms_btn_click)
dft_details_widget.calc_type.observe(lambda c: guess_calc_params(), names='value')

##NEB
neb_details_widget = NEBDetails(job_details=job_details) 

##VIEWER
viewer_widget=ViewerDetails()

mol_ids_info_out = ipw.Output()

#display(ipw.VBox([struct_browser, viewer]))
display(ipw.VBox([struct_browser, viewer_widget, mol_ids_info_out]))

#display code,dft,neb dropdown
display(ipw.VBox([computer_code_dropdown, dft_details_widget,neb_details_widget]))


In [ ]:
## Continue NEB
#
#def on_load_old_neb_press(c):
#    
#    old_neb_node = load_node(old_neb_pk_text.value)
#    
#    on_struct_change(0)
#    
#
#old_neb_pk_text = ipw.IntText(description='Old NEB PK',
#                           value=0)
#
#load_old_neb_button = ipw.Button(description='Load NEB')
#
#display(ipw.HBox([old_neb_pk_text, load_old_neb_button]))

In [ ]:
#old_neb_node = load_node(old_neb_pk_text.value)

In [ ]:
#old_neb_node.out.CALL.description

## Submission

In [ ]:

#display submit button
btn_submit = SubmitButton(the_workchain=the_workchain,job_details=job_details)
display(btn_submit)